In [1]:
%pip install datasets transformers 'accelerate>=0.26.0'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 12.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 4.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 kB 2.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 5.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 7.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 5.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from datasets import load_dataset   
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, AutoTokenizer
import numpy as np
import torch

dsn = "cubbk/zac_sample-dataset-tokenised"
model_name = "canopylabs/orpheus-tts-0.1-pretrained"

# Training Args
epochs = 1
batch_size = 1
number_processes = 1
pad_token = 128263
save_steps = 5000
learning_rate = 5.0e-5

# Naming and paths
save_folder = "checkpoints"
project_name = "tuning-orpheus"
run_name = "5e5-0"

# Ensure bf16 only when supported
bf16_supported = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
dtype = torch.bfloat16 if bf16_supported else torch.float32
if not bf16_supported:
    print("bfloat16 not supported on this device; using float32.")

# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = pad_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=dtype,
)

raw_ds = load_dataset(dsn, split="train")
split = raw_ds.train_test_split(test_size=0.05, seed=42)
train_ds, eval_ds = split["train"], split["test"]

training_args = TrainingArguments(
    overwrite_output_dir=True,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    logging_steps=1,
    bf16=bf16_supported,
    output_dir=f"./{save_folder}",
    # report_to="wandb",
    save_steps=save_steps,
    remove_unused_columns=True,
    learning_rate=learning_rate,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds, # type: ignore
    eval_dataset=eval_ds, # type: ignore
)

trainer.train()


tokenizer_config.json:   0%|          | 0.00/5.41M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/22.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

README.md:   0%|          | 0.00/344 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/96.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20 [00:00<?, ? examples/s]

Step,Training Loss
1,4.922500
2,5.116700
3,5.063200
4,4.754500
5,4.742000
6,4.598100
7,4.710000
8,4.818900
9,4.781500
10,4.756500


TrainOutput(global_step=19, training_loss=4.779965551275956, metrics={'train_runtime': 36.7189, 'train_samples_per_second': 0.517, 'train_steps_per_second': 0.517, 'total_flos': 174198588825600.0, 'train_loss': 4.779965551275956, 'epoch': 1.0})

In [2]:
import math
eval_results = trainer.evaluate()

perplexity = math.exp(eval_results["eval_loss"])
print("Eval loss:", eval_results["eval_loss"])
print("Perplexity:", perplexity)

Eval loss: 4.690624237060547
Perplexity: 108.921151209337


In [3]:
# Save final model & tokenizer
save_path = "./final-model"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('./final-model/tokenizer_config.json',
 './final-model/special_tokens_map.json',
 './final-model/chat_template.jinja',
 './final-model/tokenizer.json')

In [4]:
reloaded = AutoModelForCausalLM.from_pretrained(save_path, torch_dtype=dtype)
_ = reloaded.generate(**tokenizer("Test:", return_tensors="pt"), max_new_tokens=5)
print("Reload OK.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Reload OK.


In [6]:
prompt = "Hey, how are you doing today?"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
model.eval()
with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=50,
        temperature=0.8,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

tensor([[128000,  19182,     11,   1268,    527,    499,   3815,   3432,     30,
         128009]], device='cuda:0')


In [7]:
%pip install huggingface_hub
# huggingface-cli login
reloaded.push_to_hub("cubbk/orpheus-swedish")
tokenizer.push_to_hub("cubbk/orpheus-swedish")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.61G [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/22.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/cubbk/orpheus-swedish/commit/5121475167c2f3bb37e34448a302846b826d8e77', commit_message='Upload tokenizer', commit_description='', oid='5121475167c2f3bb37e34448a302846b826d8e77', pr_url=None, repo_url=RepoUrl('https://huggingface.co/cubbk/orpheus-swedish', endpoint='https://huggingface.co', repo_type='model', repo_id='cubbk/orpheus-swedish'), pr_revision=None, pr_num=None)